In [1]:
import librosa
import librosa.display
import tensorflow as tf
import kapre
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd
import numpy as np
import pathlib
import os
import glob
import random
import math 
import soundfile as sf
import librosa 
from plotnine import *
from sklearn.metrics import confusion_matrix, mean_squared_error
import seaborn as sn
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.utils import shuffle
from tqdm.notebook import trange, tqdm

In [2]:
seed = 27
tf.random.set_seed(seed)
random.seed(seed)
gpus = tf.config.list_physical_devices('GPU')

## need this if training on GPU
## tensorflow, get your shit together

if len(gpus)>0:

    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession

    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)

In [3]:
data_path = '/tf/data/ADReSS-IS2020'
audio_path_cc = data_path + '/Full_wave_enhanced_audio/cc/'
audio_path_cd = data_path + '/Full_wave_enhanced_audio/cd/'


text_path_cc = data_path + '/transcription/cc/'
text_path_cd = data_path + '/transcription/cd/'

# Meta Data

In [4]:
meta_cc = pd.read_csv(data_path + '/cc_meta_data.txt', sep=";", header=0, 
                      names = ['ID', 'Age', 'Gender', 'MMSE'])
meta_cd = pd.read_csv(data_path + '/cd_meta_data.txt', sep=";", header=0, 
                      names = ['ID', 'Age', 'Gender', 'MMSE'])

meta = meta_cc.assign(Group = 'cc').append(meta_cd.assign(Group = 'cd')).reset_index()
meta.MMSE = pd.to_numeric(meta.MMSE.replace(' NA', 30))
meta.ID = meta.ID.str.strip()

In [5]:
meta.sample(5)

,index,ID,Age,Gender,MMSE,Group
82,28,S124,69,female,19,cd
38,38,S055,70,male,29,cc
18,18,S024,73,female,30,cc
93,39,S138,65,female,17,cd
106,52,S154,65,female,20,cd


# Baseline 

In [6]:
import tensorflow as tf

from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
seed = 30
tf.random.set_seed(seed)
np.random.seed(seed)

In [7]:
filenames = list(glob.glob('/tf/data/ADReSS-IS2020/Full_wave_enhanced_audio/*/*'))
random.shuffle(filenames)

In [8]:
def getMelSpectrogramSamples(filename, n_mels=128, fmin=20, fmax=8000):
    y, sr = librosa.load(filename)

    ps = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, fmin=fmin, fmax=fmax, power=2)
    S_dB = librosa.power_to_db(ps, ref=np.max)
    
    #15 seconds
    window_size = 640
    window_hop = 160

    # truncate at start and end to only have windows full data
    # alternative would be to zero-pad
    start_frame = window_size 
    end_frame = window_hop * math.floor(float(S_dB.shape[1]) / window_hop)

    samples = np.array([S_dB[:, frame_idx-window_size:frame_idx] for frame_idx in range(start_frame, end_frame, window_hop)])
    
    return samples

def getSpectrogramSamples(filename):
    y, sr = librosa.load(filename)
    S = np.abs(librosa.stft(y, n_fft=512))
    S_dB = librosa.amplitude_to_db(S, ref=np.max)
    
    #15 seconds
    window_size = 640
    window_hop = 20

    # truncate at start and end to only have windows full data
    # alternative would be to zero-pad
    start_frame = window_size 
    end_frame = window_hop * math.floor(float(S_dB.shape[1]) / window_hop)

    samples = np.array([S_dB[:, frame_idx-window_size:frame_idx] for frame_idx in range(start_frame, end_frame, window_hop)])
    
    return samples

In [9]:
allSamples = []
ids = np.array([])
mmse_scores = np.array([])
classes = np.array([])

for file in tqdm(filenames):
    id = file.split('/')[-1].split('.')[0]
    mmse = meta.loc[meta.ID==id,"MMSE"].values[0]
    group = meta.loc[meta.ID==id,"Group"].values[0]
    
    samples = getMelSpectrogramSamples(file)
    allSamples.append(samples)
    
    numSamples = samples.shape[0]
    
    ids = np.append(ids, np.repeat(id, numSamples))
    mmse_scores = np.append(mmse_scores, np.repeat(mmse, numSamples))
    classes = np.append(classes, np.repeat(group, numSamples))

In [10]:
X = np.expand_dims(np.vstack(allSamples), -1)
y = mmse_scores

In [11]:
group_kfold = GroupKFold(n_splits=5)
group_kfold.get_n_splits(X, y, groups=ids)

5

In [12]:
class SquareRootScheduler:
    def __init__(self, lr=0.1):
        self.lr = lr

    def __call__(self, num_update):
        return self.lr * pow(num_update + 1.0, -0.5)
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(SquareRootScheduler(lr=0.001))

es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    patience=10, 
    restore_best_weights=True)

In [13]:
test_scores = []
history_archive = []

for train_index, test_index in tqdm(group_kfold.split(X, y, groups=ids)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train, y_train = shuffle(X_train, y_train)
    X_test, y_test = shuffle(X_test, y_test)

    model = models.Sequential([
        layers.Conv2D(filters=32, kernel_size=(3,3), strides=1, padding='same', activation='relu', name='conv1'),
        layers.MaxPooling2D(pool_size=(2,2), strides=2, name='pool1'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1)
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0),
        loss='mse',
        metrics='mse'
    )
    
    history = model.fit(
        X_train, 
        y_train, 
        epochs=1000, 
        batch_size=64,
        verbose=1,
        callbacks=[lr_callback, es_callback]
    )

    history_archive.append(history)

    rmse = np.sqrt(model.evaluate(X_test, y_test, verbose=1)[0])
    test_scores.append(rmse)
    
    tf.keras.backend.clear_session()

Epoch 1/1000
21/21 [==============================] - 3s 61ms/step - loss: 13378875.1062 - mse: 13378875.1062
Epoch 2/1000
21/21 [==============================] - 1s 59ms/step - loss: 5463.4635 - mse: 5463.4635
Epoch 3/1000
21/21 [==============================] - 1s 59ms/step - loss: 525.5567 - mse: 525.5567
Epoch 4/1000
21/21 [==============================] - 1s 59ms/step - loss: 542.1264 - mse: 542.1264
Epoch 5/1000
21/21 [==============================] - 1s 60ms/step - loss: 3222.0039 - mse: 3222.0039
Epoch 6/1000
21/21 [==============================] - 1s 60ms/step - loss: 542.0390 - mse: 542.0390
Epoch 7/1000
21/21 [==============================] - 1s 60ms/step - loss: 542.8842 - mse: 542.8842
Epoch 8/1000
21/21 [==============================] - 1s 60ms/step - loss: 525.6526 - mse: 525.6526
Epoch 9/1000
21/21 [==============================] - 1s 60ms/step - loss: 525.1750 - mse: 525.1750
Epoch 10/1000
21/21 [==============================] - 1s 60ms/step - loss: 527.1220 -

21/21 [==============================] - 1s 58ms/step - loss: 501.7155 - mse: 501.7155
Epoch 83/1000
21/21 [==============================] - 1s 58ms/step - loss: 494.7558 - mse: 494.7558
Epoch 84/1000
21/21 [==============================] - 1s 58ms/step - loss: 508.9527 - mse: 508.9527
Epoch 85/1000
21/21 [==============================] - 1s 58ms/step - loss: 512.4135 - mse: 512.4135
Epoch 86/1000
21/21 [==============================] - 1s 58ms/step - loss: 517.1299 - mse: 517.1299
Epoch 87/1000
21/21 [==============================] - 1s 58ms/step - loss: 498.3603 - mse: 498.3603
Epoch 88/1000
21/21 [==============================] - 1s 58ms/step - loss: 507.6950 - mse: 507.6950
Epoch 89/1000
21/21 [==============================] - 1s 58ms/step - loss: 515.7764 - mse: 515.7764
Epoch 90/1000
21/21 [==============================] - 1s 58ms/step - loss: 506.3708 - mse: 506.3708
Epoch 91/1000
21/21 [==============================] - 1s 58ms/step - loss: 514.1606 - mse: 514.1606
Epoc

21/21 [==============================] - 1s 58ms/step - loss: 483.8924 - mse: 483.8924
Epoch 163/1000
21/21 [==============================] - 1s 58ms/step - loss: 479.3466 - mse: 479.3466
Epoch 164/1000
21/21 [==============================] - 1s 58ms/step - loss: 480.6759 - mse: 480.6759
Epoch 165/1000
21/21 [==============================] - 1s 58ms/step - loss: 478.3165 - mse: 478.3165
Epoch 166/1000
21/21 [==============================] - 1s 58ms/step - loss: 494.1322 - mse: 494.1322
Epoch 167/1000
21/21 [==============================] - 1s 58ms/step - loss: 481.2869 - mse: 481.2869
Epoch 168/1000
21/21 [==============================] - 1s 58ms/step - loss: 489.1796 - mse: 489.1796
Epoch 169/1000
21/21 [==============================] - 1s 58ms/step - loss: 482.1797 - mse: 482.1797
Epoch 170/1000
21/21 [==============================] - 1s 58ms/step - loss: 494.8755 - mse: 494.8755
Epoch 171/1000
21/21 [==============================] - 1s 58ms/step - loss: 488.1935 - mse: 488.

21/21 [==============================] - 1s 58ms/step - loss: 478.3067 - mse: 478.3067
Epoch 243/1000
21/21 [==============================] - 1s 58ms/step - loss: 466.2091 - mse: 466.2091
Epoch 244/1000
21/21 [==============================] - 1s 58ms/step - loss: 465.7165 - mse: 465.7165
Epoch 245/1000
21/21 [==============================] - 1s 58ms/step - loss: 455.8477 - mse: 455.8477
Epoch 246/1000
21/21 [==============================] - 1s 58ms/step - loss: 469.9074 - mse: 469.9074
Epoch 247/1000
21/21 [==============================] - 1s 58ms/step - loss: 471.7052 - mse: 471.7052
Epoch 248/1000
21/21 [==============================] - 1s 58ms/step - loss: 470.7432 - mse: 470.7432
Epoch 249/1000
21/21 [==============================] - 1s 59ms/step - loss: 463.3906 - mse: 463.3906
Epoch 250/1000
21/21 [==============================] - 1s 58ms/step - loss: 466.6456 - mse: 466.6456
Epoch 251/1000
21/21 [==============================] - 1s 58ms/step - loss: 459.8261 - mse: 459.

21/21 [==============================] - 1s 58ms/step - loss: 433.4253 - mse: 433.4253
Epoch 323/1000
21/21 [==============================] - 1s 58ms/step - loss: 460.9581 - mse: 460.9581
Epoch 324/1000
21/21 [==============================] - 1s 58ms/step - loss: 455.9238 - mse: 455.9238
Epoch 325/1000
21/21 [==============================] - 1s 58ms/step - loss: 450.8176 - mse: 450.8176
Epoch 326/1000
21/21 [==============================] - 1s 58ms/step - loss: 466.4990 - mse: 466.4990
Epoch 327/1000
21/21 [==============================] - 1s 58ms/step - loss: 447.4747 - mse: 447.4747
Epoch 328/1000
21/21 [==============================] - 1s 59ms/step - loss: 449.5134 - mse: 449.5134
Epoch 329/1000
21/21 [==============================] - 1s 58ms/step - loss: 448.9797 - mse: 448.9797
Epoch 330/1000
21/21 [==============================] - 1s 58ms/step - loss: 456.1647 - mse: 456.1647
Epoch 331/1000
21/21 [==============================] - 1s 58ms/step - loss: 448.4658 - mse: 448.

...

21/21 [==============================] - 1s 57ms/step - loss: 494.1796 - mse: 494.1796
Epoch 46/1000
21/21 [==============================] - 1s 57ms/step - loss: 505.6684 - mse: 505.6684
Epoch 47/1000
21/21 [==============================] - 1s 57ms/step - loss: 494.2874 - mse: 494.2874
Epoch 48/1000
21/21 [==============================] - 1s 57ms/step - loss: 497.1171 - mse: 497.1171
Epoch 49/1000
21/21 [==============================] - 1s 57ms/step - loss: 494.3109 - mse: 494.3109
Epoch 50/1000
21/21 [==============================] - 1s 57ms/step - loss: 499.0608 - mse: 499.0608
Epoch 51/1000
21/21 [==============================] - 1s 57ms/step - loss: 501.2597 - mse: 501.2597
Epoch 52/1000
21/21 [==============================] - 1s 57ms/step - loss: 501.5862 - mse: 501.5862
Epoch 53/1000
21/21 [==============================] - 1s 57ms/step - loss: 498.1514 - mse: 498.1514
Epoch 54/1000
21/21 [==============================] - 1s 57ms/step - loss: 488.4676 - mse: 488.4676
Epoc

Epoch 126/1000
21/21 [==============================] - 1s 57ms/step - loss: 467.1648 - mse: 467.1648
Epoch 127/1000
21/21 [==============================] - 1s 57ms/step - loss: 485.9859 - mse: 485.9859
Epoch 128/1000
21/21 [==============================] - 1s 57ms/step - loss: 487.8695 - mse: 487.8695
Epoch 129/1000
21/21 [==============================] - 1s 58ms/step - loss: 475.1717 - mse: 475.1717
Epoch 130/1000
21/21 [==============================] - 1s 58ms/step - loss: 491.1506 - mse: 491.1506
Epoch 131/1000
21/21 [==============================] - 1s 57ms/step - loss: 475.3164 - mse: 475.3164
Epoch 132/1000
21/21 [==============================] - 1s 57ms/step - loss: 480.4893 - mse: 480.4893
Epoch 133/1000
21/21 [==============================] - 1s 57ms/step - loss: 484.8246 - mse: 484.8246
Epoch 134/1000
21/21 [==============================] - 1s 57ms/step - loss: 471.7040 - mse: 471.7040
Epoch 135/1000
21/21 [==============================] - 1s 57ms/step - loss: 482.7

21/21 [==============================] - 1s 57ms/step - loss: 464.1925 - mse: 464.1925
Epoch 207/1000
21/21 [==============================] - 1s 57ms/step - loss: 457.1998 - mse: 457.1998
Epoch 208/1000
21/21 [==============================] - 1s 57ms/step - loss: 465.7577 - mse: 465.7577
Epoch 209/1000
21/21 [==============================] - 1s 57ms/step - loss: 473.5820 - mse: 473.5820
Epoch 210/1000
21/21 [==============================] - 1s 57ms/step - loss: 467.0602 - mse: 467.0602
Epoch 211/1000
21/21 [==============================] - 1s 57ms/step - loss: 462.0995 - mse: 462.0995
Epoch 212/1000
21/21 [==============================] - 1s 57ms/step - loss: 475.2757 - mse: 475.2757
Epoch 213/1000
21/21 [==============================] - 1s 57ms/step - loss: 467.0372 - mse: 467.0372
Epoch 214/1000
21/21 [==============================] - 1s 57ms/step - loss: 464.9540 - mse: 464.9540
Epoch 215/1000
21/21 [==============================] - 1s 57ms/step - loss: 467.4682 - mse: 467.

21/21 [==============================] - 1s 57ms/step - loss: 463.4106 - mse: 463.4106
Epoch 287/1000
21/21 [==============================] - 1s 57ms/step - loss: 467.8576 - mse: 467.8576
Epoch 288/1000
21/21 [==============================] - 1s 57ms/step - loss: 457.9079 - mse: 457.9079
Epoch 289/1000
21/21 [==============================] - 1s 57ms/step - loss: 460.8218 - mse: 460.8218
Epoch 290/1000
21/21 [==============================] - 1s 57ms/step - loss: 454.6926 - mse: 454.6926
Epoch 291/1000
21/21 [==============================] - 1s 57ms/step - loss: 460.3917 - mse: 460.3917
Epoch 292/1000
21/21 [==============================] - 1s 57ms/step - loss: 444.2867 - mse: 444.2867
Epoch 293/1000
21/21 [==============================] - 1s 58ms/step - loss: 457.6844 - mse: 457.6844
Epoch 294/1000
21/21 [==============================] - 1s 58ms/step - loss: 453.2041 - mse: 453.2041
Epoch 295/1000
21/21 [==============================] - 1s 57ms/step - loss: 457.1507 - mse: 457.

ResourceExhaustedError:  OOM when allocating tensor with shape[655360,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node Adam/clip_by_norm_2/ArithmeticOptimizer/ReplaceMulWithSquare_mul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_78960]

Function call stack:
train_function


In [14]:
test_scores

[20.415080521401634,
 16.570791335608607,
 12.811560917376479,
 23.226446875763333]

# Save model

In [ ]:
!pwd

In [ ]:
!mkdir -p saved_model
model.save('saved_model/base_line/20210210')

In [ ]:
ls saved_model

In [ ]:
ls saved_model/base_line

In [ ]:
new_model = tf.keras.models.load_model('saved_model/base_line/20210210')
new_model.summary()

In [ ]:
loss, mse = new_model.evaluate(test_ds.batch(1), verbose=2)
print('Restored model, accuracy: {:5.2f}'.format(np.sqrt(mse)))

In [ ]:
!saved_model_cli show --dir {'saved_model/base_line/20210210'} --all

# Evaluate the performance

In [ ]:
train_files[0]

In [ ]:
new_model.predict(preprocess_dataset([train_files[0]]).batch(1), verbose=2)

In [ ]:
meta = meta.assign(split = '', predict = -99.99, predict_group = 'cc')


In [ ]:
for file in train_files:
    ID = file.numpy().decode('utf-8').split('/')[-1].split('.')[0]
    predict = model.predict(preprocess_dataset([file]).batch(1), verbose=0)
    meta.at[np.where(meta.ID == (ID + ' '))[0][0], 'predict'] = predict
    meta.at[np.where(meta.ID == (ID + ' '))[0][0], 'split'] = 'train'
    if (predict < 24):
        meta.at[np.where(meta.ID == (ID + ' '))[0][0], 'predict_group'] = 'cd'

In [ ]:
for file in test_files:
    ID = file.numpy().decode('utf-8').split('/')[-1].split('.')[0]
    predict = model.predict(preprocess_dataset([file]).batch(1), verbose=0)
    meta.at[np.where(meta.ID == (ID + ' '))[0][0], 'predict'] = predict
    meta.at[np.where(meta.ID == (ID + ' '))[0][0], 'split'] = 'test'
    if (predict < 24):
        meta.at[np.where(meta.ID == (ID + ' '))[0][0], 'predict_group'] = 'cd'

In [ ]:
for file in val_files:
    ID = file.numpy().decode('utf-8').split('/')[-1].split('.')[0]
    predict = model.predict(preprocess_dataset([file]).batch(1), verbose=0)
    meta.at[np.where(meta.ID == (ID + ' '))[0][0], 'predict'] = predict
    meta.at[np.where(meta.ID == (ID + ' '))[0][0], 'split'] = 'val'
    if (predict < 24):
        meta.at[np.where(meta.ID == (ID + ' '))[0][0], 'predict_group'] = 'cd'

In [ ]:
meta.head()

In [ ]:
meta[meta['ID']=='S129 ']

In [ ]:
def cal_rmse(g):
    rmse = np.sqrt(mean_squared_error(g['MMSE'], g['predict'] ) )
    return pd.Series(dict(rmse = rmse ))

meta.groupby('split').apply(cal_rmse).reset_index()

In [ ]:
meta.groupby('Gender').apply(cal_rmse).reset_index()

In [ ]:
ggplot(meta) + \
geom_point(aes(x = 'MMSE', y = 'predict', colour = 'split')) + \
facet_wrap('split', nrow = 1) + \
geom_abline(aes(intercept = 0, slope = 1, lty = 2)) + \
theme(figure_size = (15, 8))

In [ ]:
y_true = meta.Group
y_pred = meta.predict_group
data = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(data, columns=np.unique(y_true), index = np.unique(y_true))
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, cmap="Reds", annot=True,annot_kws={"size": 16})# font size